In [13]:
import pandas as pd
import difflib

# Alquiler

## Fase 1

Voy a dejar un poco más limpio el dataframe. Eliminaré variables que no aportan mucho (porque ya tienen un one-hot encoding, por ejemplo) y luego limpiaré aquellas observaciones que tienen `n_baths`=0 y `area`<=1).

In [14]:
dfm=pd.read_excel('data/pre-procesado/a-prov_mad-marzo24.xlsx')
dfb=pd.read_excel('data/pre-procesado/a-mun_bcn-marzo24.xlsx')
dfr=pd.read_excel('data/pre-procesado/a-rmb-sin_bcn-marzo24.xlsx')

Elimino `terrace_m2` porque no todos los pisos tienen información sobre su metraje. En cambio, sí sobre si tienen o no una terraza.

In [15]:
dfm=dfm.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)
dfb=dfb.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)
dfr=dfr.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)

In [16]:
dfm = dfm[(dfm['n_baths'] != 0) & (dfm['area'] > 1)]
dfb = dfb[(dfb['n_baths'] != 0) & (dfb['area'] > 1)]
dfr = dfr[(dfr['n_baths'] != 0) & (dfr['area'] > 1)]

print("Longitud de datos de Madrid:", len(dfm))
print("Longitud de datos de Barcelona municipio:", len(dfb))
print("Longitud de datos de la RMB sin Barcelona municipio:", len(dfr))

Longitud de datos de Madrid: 3461
Longitud de datos de Barcelona municipio: 3139
Longitud de datos de la RMB sin Barcelona municipio: 1605


In [18]:
# Define la función de similitud de texto
def texto_similar(texto1, texto2, umbral=0.95):
    """
    Compara dos cadenas de texto y devuelve True si tienen una similitud superior al umbral dado.
    """
    s = difflib.SequenceMatcher(None, texto1, texto2)
    return s.ratio() >= umbral

# Carga tus datos en un DataFrame
# Suponiendo que ya tienes cargado tu DataFrame dfr

# Convierte todos los valores de la columna 'Description' a cadenas de texto
dfr['Description'] = dfr['Description'].astype(str)

# Aplica la función texto_similar a la columna 'Description'
duplicados = []
for i, desc1 in enumerate(dfr['Description']):
    for j, desc2 in enumerate(dfr['Description']):
        if i != j and texto_similar(desc1, desc2):
            duplicados.append((i, j))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Usuari\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Usuari\AppData\Local\Temp\ipykernel_13272\3707730314.py", line 19, in <module>
    if i != j and texto_similar(desc1, desc2):
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Usuari\AppData\Local\Temp\ipykernel_13272\3707730314.py", line 7, in texto_similar
    return s.ratio() >= umbral
           ^^^^^^^^^
  File "C:\Users\Usuari\anaconda3\Lib\difflib.py", line 619, in ratio
    matches = sum(triple[-1] for triple in self.get_matching_blocks())
                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Usuari\anaconda3\Lib\difflib.py", line 454, in get_matching_blocks
    i, j, k = x = self.find_longest_match(alo, ahi, blo, bhi)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Usuari\anaconda3\Lib\dif

In [ ]:
# Obtiene los índices únicos de las filas duplicadas
indices_duplicados = set()
for dupla in duplicados:
    indices_duplicados.update(dupla)

# Obtiene los índices que no están en los duplicados
sin_duplicados = set(range(len(dfr))) - indices_duplicados

sin_duplicados=list(sin_duplicados)

Ahora elimino duplicados.

In [5]:
# Función para calcular la similitud entre dos cadenas
def similarity(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

# Umbral de similitud para 'Description'
description_threshold = 0.95

# Convertir todos los valores de la columna 'Description' a cadenas
dfm['Description'] = dfm['Description'].astype(str)
dfb['Description'] = dfb['Description'].astype(str)
dfr['Description'] = dfr['Description'].astype(str)

# Eliminar duplicados en dfm
dfm['Description_Similarity'] = dfm['Description'].apply(lambda x: similarity(x, dfm['Description']))
duplicates_dfm = dfm[(dfm['Description_Similarity'] >= description_threshold) &
                    dfm.duplicated(subset=['area', 'precio_euros'], keep='first')]
dfm.drop(duplicates_dfm.index, inplace=True)

# Eliminar duplicados en dfb
dfb['Description_Similarity'] = dfb['Description'].apply(lambda x: similarity(x, dfb['Description']))
duplicates_dfb = dfb[(dfb['Description_Similarity'] >= description_threshold) &
                    dfb.duplicated(subset=['area', 'precio_euros'], keep='first')]
dfb.drop(duplicates_dfb.index, inplace=True)

# Eliminar duplicados en dfr
dfr['Description_Similarity'] = dfr['Description'].apply(lambda x: similarity(x, dfr['Description']))
duplicates_dfr = dfr[(dfr['Description_Similarity'] >= description_threshold) &
                    dfr.duplicated(subset=['area', 'precio_euros'], keep='first')]
dfr.drop(duplicates_dfr.index, inplace=True)

# Eliminar la columna de similitud agregada
for df in [dfm, dfb, dfr]:
    df.drop(columns=['Description_Similarity'], inplace=True)

# Verificar la longitud de los DataFrames resultantes
print("Longitud de dfm después de eliminar duplicados:", len(dfm))
print("Longitud de dfb después de eliminar duplicados:", len(dfb))
print("Longitud de dfr después de eliminar duplicados:", len(dfr))

Longitud de dfm después de eliminar duplicados: 3645
Longitud de dfb después de eliminar duplicados: 3415
Longitud de dfr después de eliminar duplicados: 1646


## Multifamiliares

In [6]:
dfm = dfm[dfm['multifam'] == 1]
dfb = dfb[dfb['multifam'] == 1]
dfr = dfr[dfr['multifam'] == 1]
print("Longitud de dfm multifam:", len(dfm))
print("Longitud de dfb multifam:", len(dfb))
print("Longitud de dfr multifam:", len(dfr))

Longitud de dfm multifam: 2841
Longitud de dfb multifam: 2965
Longitud de dfr multifam: 1020


## Etiquetas EPC

Reviso cuántos pisos tienen información de etiquetas EPC.

In [7]:
# Contar cuántas filas tienen valor 1 en la columna 'dummy_consum_EPC' de cada DataFrame
filas_con_1_consum_EPC_dfm = (dfm['dummy_consum_EPC'] == 1).sum()/len(dfm)*100
filas_con_1_consum_EPC_dfb = (dfb['dummy_consum_EPC'] == 1).sum()/len(dfb)*100
filas_con_1_consum_EPC_dfr = (dfr['dummy_consum_EPC'] == 1).sum()/len(dfr)*100

# Contar cuántas filas tienen valor 1 en la columna 'dummy_emission_EPC' de cada DataFrame
filas_con_1_emission_EPC_dfm = (dfm['dummy_emission_EPC'] == 1).sum()/len(dfm)*100
filas_con_1_emission_EPC_dfb = (dfb['dummy_emission_EPC'] == 1).sum()/len(dfb)*100
filas_con_1_emission_EPC_dfr = (dfr['dummy_emission_EPC'] == 1).sum()/len(dfr)*100

# Contar cuántas filas tienen valor 1 en ambas columnas 'dummy_consum_EPC' y 'dummy_emission_EPC' de cada DataFrame
filas_con_1_en_ambas_dfm = ((dfm['dummy_consum_EPC'] == 1) & (dfm['dummy_emission_EPC'] == 1)).sum()/len(dfm)*100
filas_con_1_en_ambas_dfb = ((dfb['dummy_consum_EPC'] == 1) & (dfb['dummy_emission_EPC'] == 1)).sum()/len(dfb)*100
filas_con_1_en_ambas_dfr = ((dfr['dummy_consum_EPC'] == 1) & (dfr['dummy_emission_EPC'] == 1)).sum()/len(dfr)*100

# Crear una tabla para mostrar el conteo por cada DataFrame
conteo_por_df = pd.DataFrame({
    'DataFrame': ['dfm', 'dfb', 'dfr'],
    'Total datos':[len(dfm),len(dfb),len(dfr)],
    'consum_EPC': [filas_con_1_consum_EPC_dfm, filas_con_1_consum_EPC_dfb, filas_con_1_consum_EPC_dfr],
    'emission_EPC': [filas_con_1_emission_EPC_dfm, filas_con_1_emission_EPC_dfb, filas_con_1_emission_EPC_dfr],
    'Ambas': [filas_con_1_en_ambas_dfm, filas_con_1_en_ambas_dfb, filas_con_1_en_ambas_dfr]
})

# Mostrar la tabla
print(conteo_por_df)

  DataFrame  Total datos  consum_EPC  emission_EPC      Ambas
0       dfm         2841   97.008096     97.008096  97.008096
1       dfb         2965   97.740304     97.740304  97.740304
2       dfr         1020   98.431373     98.431373  98.431373


En general, se observa que una gran proporción de pisos tienen información de su etiqueta EPC.

A continuación reemplazaré las filas vacías de las columnas `kwhm2_year` y `kgCO2m2_year` por 0 para luego poder identificar qué observaciones podrían no tener cargada correctamente la info sobre su rendimiento energético.

In [8]:
# Para dfm
dfm.loc[:, 'kwhm2_year'] = dfm['kwhm2_year'].fillna(0)
dfm.loc[:, 'kgCO2m2_year'] = dfm['kgCO2m2_year'].fillna(0)

# Para dfb
dfb.loc[:, 'kwhm2_year'] = dfb['kwhm2_year'].fillna(0)
dfb.loc[:, 'kgCO2m2_year'] = dfb['kgCO2m2_year'].fillna(0)

# Para dfr
dfr.loc[:, 'kwhm2_year'] = dfr['kwhm2_year'].fillna(0)
dfr.loc[:, 'kgCO2m2_year'] = dfr['kgCO2m2_year'].fillna(0)

De los que tienen información, cuántos tienen valores inválidos (999, 0).

In [9]:
# Calcular el porcentaje de filas donde 'kwhm2_year' tiene valor 999, 0 o está vacía respecto a 'dummy_consum_EPC' con valor 1 para cada DataFrame
porcentaje_kwhm2_999_consum_dfm = ((dfm['dummy_consum_EPC'] == 1) & (dfm['kwhm2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kwhm2_999_consum_dfb = ((dfb['dummy_consum_EPC'] == 1) & (dfb['kwhm2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kwhm2_999_consum_dfr = ((dfr['dummy_consum_EPC'] == 1) & (dfr['kwhm2_year'].isin([999, 0, '']))).mean() * 100

# Calcular el porcentaje de filas donde 'kgCO2m2_year' tiene valor 999, 0 o está vacía respecto a 'dummy_emission_EPC' con valor 1 para cada DataFrame
porcentaje_kgCO2m2_999_emission_dfm = ((dfm['dummy_emission_EPC'] == 1) & (dfm['kgCO2m2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kgCO2m2_999_emission_dfb = ((dfb['dummy_emission_EPC'] == 1) & (dfb['kgCO2m2_year'].isin([999, 0, '']))).mean() * 100
porcentaje_kgCO2m2_999_emission_dfr = ((dfr['dummy_emission_EPC'] == 1) & (dfr['kgCO2m2_year'].isin([999, 0, '']))).mean() * 100

# Crear una tabla para mostrar los porcentajes por cada DataFrame
porcentaje_por_df = pd.DataFrame({
    'DataFrame': ['dfm', 'dfb', 'dfr'],
    "999 consum": [porcentaje_kwhm2_999_consum_dfm, porcentaje_kwhm2_999_consum_dfb, porcentaje_kwhm2_999_consum_dfr],
    "999 emission": [porcentaje_kgCO2m2_999_emission_dfm, porcentaje_kgCO2m2_999_emission_dfb, porcentaje_kgCO2m2_999_emission_dfr]
})

# Mostrar la tabla
print(porcentaje_por_df)

  DataFrame  999 consum  999 emission
0       dfm   77.120732     97.008096
1       dfb   73.389545     97.740304
2       dfr   76.274510     98.431373


En general, el porcentaje es bastante alto, por lo que no quitaré esas filas, sólo aquellas que no tienen la letra de su etiqueta EPC.

En el código que viene a continuación lo había hecho pero lo dejo como comentario.

In [10]:
# Eliminar filas con valor 0 en 'dummy_consum_EPC' y/o 'dummy_emission_EPC'
dfm = dfm[~((dfm['dummy_consum_EPC'] == 0) | (dfm['dummy_emission_EPC'] == 0))]
dfb = dfb[~((dfb['dummy_consum_EPC'] == 0) | (dfb['dummy_emission_EPC'] == 0))]
dfr = dfr[~((dfr['dummy_consum_EPC'] == 0) | (dfr['dummy_emission_EPC'] == 0))]

# Eliminar filas con valores 999, 0 o vacíos en 'kgCO2m2_year' y/o 'kwhm2_year'
#dfm = dfm[~((dfm['kwhm2_year'].isin([999, 0])) | (dfm['kgCO2m2_year'].isin([999, 0])))]
#dfb = dfb[~((dfb['kwhm2_year'].isin([999, 0])) | (dfb['kgCO2m2_year'].isin([999, 0])))]
#dfr = dfr[~((dfr['kwhm2_year'].isin([999, 0])) | (dfr['kgCO2m2_year'].isin([999, 0])))]

# Mostrar la longitud de cada DataFrame después de eliminar las filas
#print("Longitud de dfm después de eliminar filas:", len(dfm))
#print("Longitud de dfb después de eliminar filas:", len(dfb))
#print("Longitud de dfr después de eliminar filas:", len(dfr))

In [11]:
dfm.to_excel('data/a-prov_mad-marzo24.xlsx', index=False)
#Estos necesitarán un paso más, por eso los mantengo en pre-procesado
dfb.to_excel('data/pre-procesado/a-mun_bcn-marzo24-1.xlsx', index=False)
dfr.to_excel('data/pre-procesado/a-rmb-sin_bcn-marzo24-1.xlsx', index=False)